# Contact Information: Daniel Wieferich (dwieferich@usgs.gov)                                                                                                                                                                                                                       

##### Date: 201705

# Testing pulling data from a source (this example uses the included GeoJson file) and inserting data into a database using the sql api in GC2.  This code is in progress.

In [1]:
import geopandas as gpd
import getpass
import pandas as pd
import numpy as np
import requests

In [11]:
#Import data from geoJson file, create geodataframe
gdf = gpd.read_file("insertTest.geojson")
gdf.head()

,featureuri,ftypeb,ftyped,geometry,sfrspatcert,title
0,00002971-ed66-11e6-b167-1fcf3bf50251,https://www.sciencebase.gov/vocab/term/58a487e...,https://www.sciencebase.gov/vocab/term/58a49ca...,POINT Z (-114.5478015909999 38.00542046900006 0),https://www.sciencebase.gov/vocab/term/5822257...,20595156
1,00002972-ed66-11e6-b168-8fc1348c3a58,https://www.sciencebase.gov/vocab/term/58a487e...,https://www.sciencebase.gov/vocab/term/58a49ca...,POINT Z (-114.5780308659999 38.00019370800004 0),https://www.sciencebase.gov/vocab/term/5822257...,20595158
2,00003e24-ed64-11e6-b344-0f2972444f8e,https://www.sciencebase.gov/vocab/term/58a487e...,https://www.sciencebase.gov/vocab/term/58a49ca...,POINT Z (-109.2616298429999 46.87911595100007 0),https://www.sciencebase.gov/vocab/term/5822257...,12380253


In [15]:
#Loop through rows in geodataframe and build url (WITHOUT API)
for index, row in gdf.iterrows():
    point = str(row['geometry'])  
    #Note that gc2 forced geojson input to be a multipoint, the data is actually point, geometry value needs to be changed to MultiPOINT which is reflected below
    query = "https://gc2.mapcentia.com/api/v1/sql/bcb?q=insert into sfr.sfr_pointtest (featureuri, title, sfrspatcert, ftypeb, ftyped, the_geom) VALUES ('" + row['featureuri'] + "','" + row['title'] + "','" + row['sfrspatcert'] + "','" + row['ftypeb'] + "','" +row['ftyped']+ "', ST_GeomFromText('Multi" + point+ "',4269))" 
    print query

https://gc2.mapcentia.com/api/v1/sql/bcb?q=insert into sfr.sfr_pointtest (featureuri, title, sfrspatcert, ftypeb, ftyped, the_geom) VALUES ('00002971-ed66-11e6-b167-1fcf3bf50251','20595156','https://www.sciencebase.gov/vocab/term/5822257ee4b0b3d9add24304','https://www.sciencebase.gov/vocab/term/58a487eee4b0f974afcf03b2','https://www.sciencebase.gov/vocab/term/58a49caee4b0f974afcf03b3', ST_GeomFromText('MultiPOINT Z (-114.5478015909999 38.00542046900006 0)',4269))
https://gc2.mapcentia.com/api/v1/sql/bcb?q=insert into sfr.sfr_pointtest (featureuri, title, sfrspatcert, ftypeb, ftyped, the_geom) VALUES ('00002972-ed66-11e6-b168-8fc1348c3a58','20595158','https://www.sciencebase.gov/vocab/term/5822257ee4b0b3d9add24304','https://www.sciencebase.gov/vocab/term/58a487eee4b0f974afcf03b2','https://www.sciencebase.gov/vocab/term/58a49caee4b0f974afcf03b3', ST_GeomFromText('MultiPOINT Z (-114.5780308659999 38.00019370800004 0)',4269))
https://gc2.mapcentia.com/api/v1/sql/bcb?q=insert into sfr.sfr_p

In [ ]:
#Same as previous cell, but this uses requests to submit insert to GC2 (this works)
apiKey = getpass.getpass('ApiKey: ')
for index, row in gdf.iterrows():
    try:
        point = str(row['geom'])
        #Note that gc2 forced geojson input to be a multipoint, that needs to be reflected here
        query = "https://gc2.mapcentia.com/api/v1/sql/bcb?q=insert into sfr.sfr_pointtest (featureuri, title, sfrspatcert, ftypeb, ftyped, the_geom) VALUES ('" + row['featureuri'] + "','" + row['title'] + "','" + row['sfrspatcert'] + "','" + row['ftypeb'] + "','" +row['ftyped']+ "', ST_GeomFromText('Multi" + point+ "',4269))&key=" + apiKey 
        r = requests.get(query)
        print r.json
        #print r.url
    except requests.exceptions.RequestException as e:
        print e

In [ ]:
#Using a post the following initial test was unsucessful, returning status_code 500
apiKey = getpass.getpass('ApiKey: ')
for index, row in gdf.iterrows():
    try:
        point = str(row['geom'])
        q = "insert into sfr.sfr_pointtest (featureuri, title, sfrspatcert, ftypeb, ftyped, the_geom) VALUES ('" + row['featureuri'] + "','" + row['title'] + "','" + row['sfrspatcert'] + "','" + row['ftypeb'] + "','" +row['ftyped']+ "', ST_GeomFromText('Multi" + point+ "',4269))"
        payload = {'key': apiKey, 'q':q}
        r = requests.post('https://gc2.mapcentia.com/api/v1/sql/bcb', params=payload)
        #print r.url
        print r.json
    except requests.exceptions.RequestException as e:
        print e